In [ ]:
import json
from collections import defaultdict

# 定义函数：计算节点分数
def calculate_node_ranks(evaluate_result):
    leaf_positions = []
    leaf_scores = []
    original_scores = {}

    # 提取叶子节点的位置信息和分数
    for node_id, node_data in evaluate_result.items():
        if all(k in node_data for k in ['stage_1', 'stage_2', 'stage_3', 'total_score']):
            pos = (int(node_data['stage_1']), int(node_data['stage_2']), int(node_data['stage_3']))
            score = float(node_data['total_score'])
            leaf_positions.append(pos)
            leaf_scores.append(score)
            original_scores[pos] = score

    if not leaf_scores:
        return {"root": 0, "level_1": {}, "level_2": {}, "level_3": {}}

    # 排名传导
    sorted_idx = sorted(range(len(leaf_scores)), key=lambda i: leaf_scores[i], reverse=True)
    ranked_scores = [0] * len(leaf_scores)
    for rank, idx in enumerate(sorted_idx, start=1):
        if rank <= 10:
            ranked_scores[idx] = 2
        elif rank <= 20:
            ranked_scores[idx] = 1
        elif rank <= 40:
            ranked_scores[idx] = 0
        elif rank <= 50:
            ranked_scores[idx] = -1
        else:
            ranked_scores[idx] = -2

    # 构建树状分数
    tree_scores = {}
    for i, pos in enumerate(leaf_positions):
        tree_scores[pos] = ranked_scores[i]

    # 计算 level 2 和 level 1
    level2_children = defaultdict(list)
    level1_nodes = set()
    level2_nodes = set()
    for pos in leaf_positions:
        s1, s2, _ = pos
        level1_nodes.add((s1,))
        level2_nodes.add((s1, s2))
        level2_children[(s1, s2)].append(tree_scores[pos])
    for parent, scores in level2_children.items():
        tree_scores[parent] = sum(scores) / len(scores)
    level1_children = defaultdict(list)
    for s1, s2 in level2_nodes:
        level1_children[(s1,)].append(tree_scores[(s1, s2)])
    for parent, scores in level1_children.items():
        tree_scores[parent] = sum(scores) / len(scores)
    # 根节点
    tree_scores[()] = sum(tree_scores[pos] for pos in level1_nodes) / len(level1_nodes)

    # 构造结构化输出
    node_ranks = {
        "root": round(tree_scores.get((), 0), 4),
        "level_1": {str(s1): round(tree_scores[(s1,)], 4) for (s1,) in sorted(level1_nodes)},
        "level_2": {f"{s1}_{s2}": round(tree_scores[(s1, s2)], 4) for (s1, s2) in sorted(level2_nodes)},
        "level_3": {f"{s1}_{s2}_{s3}": round(original_scores[(s1, s2, s3)], 4)
                    for (s1, s2, s3) in sorted(leaf_positions)}
    }
    return node_ranks

# 定义函数：选择 Stage 1 对
def select_stage1(level1):
    sorted_nodes = sorted(level1.items(), key=lambda x: x[1], reverse=True)
    keys = [k for k, _ in sorted_nodes]
    highest = keys[0]
    lowest = keys[-1]
    second_lowest = keys[-2] if len(keys) >= 2 else lowest
    return [(highest, lowest), (highest, second_lowest)]

# 加载 JSON
file_path = '/mnt/yuhao/SuperWrite/Data/DPO-data/Final_version_inference_save_2/query_1.json'
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 提取 evaluate_result 和计算 node_ranks
evaluate_result = data.get('evaluate_result', {})
node_ranks = calculate_node_ranks(evaluate_result)

# 获取 Stage 1 输入和输出
stage1_input = data.get('Stage_1_input', '')
stage1_outputs = data.get('Stage_1_output', [])

# 生成 Stage 1 对
stage1_pairs = select_stage1(node_ranks['level_1'])

# 构建 DPO 数据列表
dpo_data = []
for chosen_key, rejected_key in stage1_pairs:
    chosen_val = next((o['response'] for o in stage1_outputs if o.get('stage_1') == int(chosen_key)), "")
    rejected_val = next((o['response'] for o in stage1_outputs if o.get('stage_1') == int(rejected_key)), "")
    dpo_data.append({
        "conversations": [
            {"from": "human", "value": stage1_input}
        ],
        "chosen": {"from": "gpt", "value": chosen_val},
        "rejected": {"from": "gpt", "value": rejected_val}
    })

# 输出 JSON 数组
print(json.dumps(dpo_data, ensure_ascii=False, indent=2))


In [ ]:
import json
import os
import glob
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor
from json import JSONDecodeError

# 计算节点分数函数
def calculate_node_ranks(evaluate_result):
    leaf_positions = []
    leaf_scores = []
    original_scores = {}

    # 提取叶子节点的位置信息和得分
    for node_id, node_data in evaluate_result.items():
        if all(k in node_data for k in ['stage_1', 'stage_2', 'stage_3', 'total_score']):
            pos = (int(node_data['stage_1']), int(node_data['stage_2']), int(node_data['stage_3']))
            score = float(node_data['total_score'])
            leaf_positions.append(pos)
            leaf_scores.append(score)
            original_scores[pos] = score

    if not leaf_scores:
        return {"root": 0, "level_1": {}, "level_2": {}, "level_3": {}}

    # 基于比值分配传导分
    total = len(leaf_scores)
    tree_scores = {}
    for rank, idx in enumerate(sorted(range(total), key=lambda i: leaf_scores[i], reverse=True), start=1):
        ratio = rank / total
        if ratio <= 1/6:
            score_val = 2
        elif ratio <= 2/6:
            score_val = 1
        elif ratio <= 4/6:
            score_val = 0
        elif ratio <= 5/6:
            score_val = -1
        else:
            score_val = -2
        tree_scores[leaf_positions[idx]] = score_val

    # 计算 level_2 和 level_1 节点分数
    level2_children = defaultdict(list)
    level1_nodes = set()
    level2_nodes = set()
    for pos, score in tree_scores.items():
        if isinstance(pos, tuple) and len(pos) == 3:
            s1, s2 = pos[0], pos[1]
            level1_nodes.add((s1,))
            level2_nodes.add((s1, s2))
            level2_children[(s1, s2)].append(score)

    for parent, scores in level2_children.items():
        tree_scores[parent] = sum(scores) / len(scores)

    level1_children = defaultdict(list)
    for s1, s2 in level2_nodes:
        level1_children[(s1,)].append(tree_scores[(s1, s2)])
    for parent, scores in level1_children.items():
        tree_scores[parent] = sum(scores) / len(scores)

    # 根节点分数
    tree_scores[()] = (sum(tree_scores[p] for p in level1_nodes) / len(level1_nodes)) if level1_nodes else 0

    # 组织输出
    node_ranks = {
        "root": round(tree_scores.get((), 0), 4),
        "level_1": {str(s1): round(tree_scores[(s1,)], 4) for (s1,) in sorted(level1_nodes)},
        "level_2": {f"{s1}_{s2}": round(tree_scores[(s1, s2)], 4) for (s1, s2) in sorted(level2_nodes)},
        "level_3": {f"{s1}_{s2}_{s3}": round(original_scores[(s1, s2, s3)], 4) for (s1, s2, s3) in sorted(leaf_positions)}
    }
    return node_ranks

# 选择 Stage 对策略
def select_stage1(level1):
    sorted_nodes = sorted(level1.items(), key=lambda x: x[1], reverse=True)
    keys = [k for k, _ in sorted_nodes]
    if not keys:
        return []
    highest = keys[0]
    if len(keys) == 1:
        # 只有一个节点时，只返回一个对
        return [(highest, highest)]
    lowest = keys[-1]
    second_lowest = keys[-2]
    return [(highest, lowest), (highest, second_lowest)]

def select_stage2(level2, parents):
    groups = []
    for parent in parents:
        children = {k: v for k, v in level2.items() if k.startswith(parent + '_')}
        if children:
            sorted_children = sorted(children.items(), key=lambda x: x[1], reverse=True)
            groups.append((sorted_children[0][0], sorted_children[-1][0]))
    return groups


def select_stage3(level3, parents):
    groups = []
    for parent in parents:
        children = {k: v for k, v in level3.items() if k.startswith(parent + '_')}
        if children:
            sorted_children = sorted(children.items(), key=lambda x: x[1], reverse=True)
            groups.append((sorted_children[0][0], sorted_children[-1][0]))
    return groups

# Helper: 查找输入/输出
def find_input(inputs, **criteria):
    for inp in inputs:
        if all(inp.get(k) == v for k, v in criteria.items()):
            return inp.get('response', '')
    return ''

def find_output(outputs, **criteria):
    for out in outputs:
        if all(out.get(k) == v for k, v in criteria.items()):
            return out.get('response', '')
    return ''

# 处理单个文件
def process_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except JSONDecodeError as e:
        print(f"跳过文件 {file_path}，JSON 解析失败: {e}")
        return []

    evaluate_result = data.get('evaluate_result', {})
    node_ranks = calculate_node_ranks(evaluate_result)
    dpo_data = []

    # Stage1
    stage1_input = data.get('Stage_1_input', '')
    for chosen, rejected in select_stage1(node_ranks['level_1']):
        cv = find_output(data.get('Stage_1_output', []), stage_1=int(chosen))
        rv = find_output(data.get('Stage_1_output', []), stage_1=int(rejected))
        dpo_data.append({
            "conversations": [{"from": "human", "value": stage1_input}],
            "chosen": {"from": "gpt", "value": cv},
            "rejected": {"from": "gpt", "value": rv}
        })

    # Stage2
    lvl1_sorted = sorted(node_ranks['level_1'].items(), key=lambda x: x[1], reverse=True)
    parents2 = [lvl1_sorted[i][0] for i in range(min(2, len(lvl1_sorted)))]
    for chosen, rejected in select_stage2(node_ranks['level_2'], parents2):
        s1, s2 = map(int, chosen.split('_'))
        _, r2 = map(int, rejected.split('_'))
        inp = find_input(data.get('Stage_2_input', []), stage_1=s1)
        cv = find_output(data.get('Stage_2_output', []), stage_1=s1, stage_2=s2)
        rv = find_output(data.get('Stage_2_output', []), stage_1=s1, stage_2=r2)
        dpo_data.append({
            "conversations": [{"from": "human", "value": inp}],
            "chosen": {"from": "gpt", "value": cv},
            "rejected": {"from": "gpt", "value": rv}
        })

    # Stage3
    parents3 = [ch for ch, _ in select_stage2(node_ranks['level_2'], parents2)]
    for chosen, rejected in select_stage3(node_ranks['level_3'], parents3):
        s1, s2, s3 = map(int, chosen.split('_'))
        _, _, r3 = map(int, rejected.split('_'))
        inp = find_input(data.get('Stage_3_input', []), stage_1=s1, stage_2=s2)
        cv = find_output(data.get('Stage_3_output', []), stage_1=s1, stage_2=s2, stage_3=s3)
        rv = find_output(data.get('Stage_3_output', []), stage_1=s1, stage_2=s2, stage_3=r3)
        dpo_data.append({
            "conversations": [{"from": "human", "value": inp}],
            "chosen": {"from": "gpt", "value": cv},
            "rejected": {"from": "gpt", "value": rv}
        })

    return dpo_data

# 主函数：并发处理文件夹下所有 JSON，输出扁平列表并打印总量
if __name__ == '__main__':
    folder = ''
    file_list = glob.glob(os.path.join(folder, '*.json'))

    all_dpo = []
    with ProcessPoolExecutor(max_workers=256) as executor:
        for result in executor.map(process_file, file_list):
            if result:
                all_dpo.extend(result)

    # 保存为扁平 JSON 数组
    out_path = os.path.join(folder, 'aggregated_dpo_data.json')
    with open(out_path, 'w', encoding='utf-8') as fw:
        json.dump(all_dpo, fw, ensure_ascii=False, indent=2)

    print(f"Aggregated DPO data saved to: {out_path}")
    print(f"Total DPO pairs: {len(all_dpo)}")
